<a href="https://colab.research.google.com/github/1306minyoung/data/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**일반적인 모델 생성 방법**

1. 훈련 세트로 모델을 생성 - 하이퍼 파라미터 조절

2. 테스트 세트로 모델을 평가 - 성능이 만족스러우면 모델 생성 완료(맘에 안들면 반복)

이러한 과정을 반복하다보면 결국 모델이 훈련세트뿐만 아니라 테스트 세트에도 맞추어짐

**검증세트**

모델을 만들고나서 테스트세트는 딱 1번 사용되는것이 좋으니 추가로 검증세트를 또 나눔. 대략 훈련60, 검증20, 테스트20

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


##**교차검증**

검증세트를 뗴어내어 평가하는 과정을 여러번 반복함

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02133226, 0.03201818, 0.03466606, 0.04121399, 0.03095055]), 'score_time': array([0.00612879, 0.02087975, 0.00202203, 0.00622892, 0.01200652]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score'])) #현 모델의 검증 점수

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
print(splitter)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(scores)
print(np.mean(scores['test_score']))

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
{'fit_time': array([0.01809597, 0.01218534, 0.0159924 , 0.01225328, 0.01197124,
       0.01206875, 0.01210713, 0.01205516, 0.02215457, 0.01182723]), 'score_time': array([0.00180507, 0.00177717, 0.00173092, 0.00175333, 0.00149965,
       0.00172496, 0.00198984, 0.00169778, 0.00170898, 0.00175881]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
0.8574181117533719


##하이퍼 파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#n_jobs : 병렬 실행에서 사용할 CPU 코어 수. -1이면 모든 코어 사용

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_ #검증점수가 가장 높은 매개변수 조합의 모델은 gs.best_estimator_속성에 저장
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) #np argmax함수를 이용해 가장 큰 값의 인덱스를 추
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [19]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [24]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


##랜덤서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있고,

너무 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래걸릴 수 있음

==> 랜덤 서치 사용

In [25]:
from scipy.stats import uniform, randint

In [26]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 0, 1, 6, 4, 4, 5, 3, 7, 0])

10번의 샘플링을 했기에 고르게 샘플링되는 것 같아보이지 않음.

그러나 샘플링 숫자를 늘리면 쉽게 확인 가능

In [27]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 101, 103, 109,  94, 113,  98,  88, 101,  90]))

In [28]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.9305319 , 0.57533363, 0.90610133, 0.27893981, 0.89687858,
       0.13442799, 0.91477193, 0.96181078, 0.19472175, 0.10771173])

샘플링 횟수는 n_iter 매개변수에 지정

In [30]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c0ec37bc5e0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c0ec37bfa30>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c0ec37bc880>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c0ec37bf370>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


##주의
교차검증, 그리드 서치, 랜덤서치는 모델훈련이 아닌 검증.

즉, 적절한 하이퍼파라미터를 찾는데 설명해야함


In [ ]:
#그리드서치 || 랜덤서치를 사용해 최적화하기
